# <center>Исследование SQL запросов</center>

#### Цели исследования:
* На основе предоставленных данных из БД конкурирующего сервиса, сформировать ценностное предложение нового приложения для любителей книг

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

Создадим коннекцию к базе, предварительно сохранив файл сертификата в папке с проектом:

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Обзор таблиц

#### Таблица `books` содержит данные о книгах

In [3]:
pd.io.sql.read_sql('SELECT * FROM books LIMIT 5', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.
___

#### Таблица `authors` содержит данные об авторах

In [4]:
pd.io.sql.read_sql('SELECT * FROM authors LIMIT 5', con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


- `author_id` — идентификатор автора;
- `author` — имя автора.
___

#### Таблица `publishers` содержит данные об издательствах

In [5]:
pd.io.sql.read_sql('SELECT * FROM publishers LIMIT 5', con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;
___

#### Таблица `ratings` содержит данные о пользовательских оценках книг

In [6]:
pd.io.sql.read_sql('SELECT * FROM ratings LIMIT 5', con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.
___

#### Таблица `reviews` cодержит данные о пользовательских обзорах на книги

In [7]:
pd.io.sql.read_sql('SELECT * FROM reviews LIMIT 5', con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Посчитаем количество книг, выпущенных после 1 января 2000 года

In [8]:
query = '''
SELECT
    COUNT(book_id) as book_cnt
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''

In [9]:
pd.io.sql.read_sql(query, con = engine)

,book_cnt
0,819


В базе хранятся данные о 819 книгах, выпушенных с 1 января 2000 года. 

### Посчитаем количество пользовательских обзоров и среднюю оценку для каждой книги;

In [10]:
query = '''
SELECT
    books.book_id,
    books.title,
    COUNT(DISTINCT review_id) as review_cnt,
    AVG(rating) as avg_rating
FROM 
    books
INNER JOIN reviews ON  reviews.book_id = books.book_id
INNER JOIN ratings ON  ratings.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    review_cnt DESC
LIMIT 10
'''

In [11]:
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_cnt,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,207,Eat Pray Love,6,3.395833
3,627,The Alchemist,6,3.789474
4,673,The Catcher in the Rye,6,3.825581
5,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,656,The Book Thief,6,4.264151


In [12]:
pd.io.sql.read_sql(query, con = engine).review_cnt.sum()

61

Больше всего пользовательский обзоров (7) было написано для книги "Сумерки"

### Определим издательство, которое издало наибольшее число книг толще 50 страниц

In [13]:
query = '''
SELECT
    publisher,
    COUNT(book_id) as book_cnt
FROM
    publishers
LEFT JOIN books ON  books.publisher_id = publishers.publisher_id 
WHERE 
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    book_cnt DESC
'''

In [14]:
pd.io.sql.read_sql(query, con = engine)

,publisher,book_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Британское издательство "Penguin Books", основанное в 1935 году, в данной выборке имеет на счету 42 выпущенных книги. Издательство, выпустившее такие популярные книги как "О мышах и людях" - Джона Стейнбека, "Заводной апельсин" - Энтони Бёрджесса и множество других. 

### Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками

In [15]:
query = '''
SELECT
    author,
    AVG(sub.avg_rating) as total_avg_rating
FROM
    (SELECT
        author,
        books.book_id,
        AVG(rating) as avg_rating
    FROM 
        authors
    LEFT JOIN books ON books.author_id = authors.author_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        author,
        books.book_id
    HAVING
        COUNT(rating) > 50) as sub
GROUP BY
    author
ORDER BY
    total_avg_rating DESC
'''

In [16]:
pd.io.sql.read_sql(query, con = engine)

,author,total_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самый высокий средний рейтинг (4,3) в данный выборке у писательницы Джоан Роулинг, наиболее известная как автор серии романов о Гарри Поттере. 

### Посчитаем среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок

In [17]:
query = '''
SELECT
    ROUND(AVG(sub_text_cnt.text_cnt)) AS avg_text_cnt
FROM
    (SELECT
        COUNT(text) as text_cnt
    FROM 
        reviews
    WHERE username IN
        (SELECT username
        FROM 
            ratings
        GROUP BY
            username
        HAVING
            COUNT(ratings) > 50)
    GROUP BY 
            username
    ) AS sub_text_cnt  
'''

In [18]:
pd.io.sql.read_sql(query, con = engine)

,avg_text_cnt
0,24.0


## Выводы:

По итогам запросов для данных из предоставленной БД:
* После 1 января 2000 года было выпущенно 819 книг
* Книга "Сумерки" насчитывает больше всего пользовательских обзоров (1120) и имеет среднюю оценку в	3.6 балла
* Издательство Penguin Books выпустило больше всех книг (42 книги)
* Автор с самой высокой средней оценкой книг - Джоан Роулинг
* В среднем, пользователь который поставил больше 50 оценок, пишет 24 текстовых обзора. 